Figures S3 to S49 of Tsutsui (2020, https://doi.org/10.1029/2019GL085844) including updates

In [2]:
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from mce.core.forcing import RfCO2
from mce.core.climate import IrmBase
from mce.util.plot_base import PlotBase
from mce.util.plotfunc import plot_fitting

In [3]:
# Normalized CMIP data
with pd.HDFStore('data/cmip_normalized.h5', 'r') as store:
    df_norm = {
        'CMIP5':
        store['CMIP5/climate_norm1']
        .rename({'abrupt4xCO2': 'abrupt-4xCO2'}),
        'CMIP6':
        store['CMIP6/climate_norm1'],
    }

In [4]:
# Calibrated thermal parameters
use_new_data = True

if use_new_data:
    outpath = 'data/parms_calib_climate.h5'
    with pd.HDFStore(outpath, 'r') as store:
        df_parms = (
            store['cmip_norm1__nl3'] # 3-layer model
            .sort_index(key=lambda x: x.str.lower())
        )
        df_parms['ecs_reg'] = 0.5 * df_parms['q4x_reg'] / df_parms['lambda_reg']

else:
    from netCDF4 import Dataset

    df_parms = {}

    with Dataset('mce/data/parms/parms_irm-3_rtnt-tas_cmip5.nc') as ncf:
        df_parms['CMIP5'] = pd.DataFrame({
            k: v[:].filled()
            for k, v in ncf.variables.items() if k not in ['dataset']
        }, index=[
            ''.join(x.astype(str)).strip()
            for x in ncf.variables['dataset'][:]
        ])

    with Dataset('mce/data/parms/parms_irm-3_rtnt-tas_cmip6.nc') as ncf:
        df_parms['CMIP6'] = pd.DataFrame({
            k: v[:].filled()
            for k, v in ncf.variables.items() if k not in ['dataset']
        }, index=[
            ''.join(x.astype(str)).strip()
            for x in ncf.variables['dataset'][:]
        ])
    df_parms = pd.concat(df_parms).rename(
        columns={
            'alpha': 'co2_alpha',
            'beta': 'co2_beta',
            'time_constant_0': 'tau0',
            'time_constant_1': 'tau1',
            'time_constant_2': 'tau2',
            'amplitude_0': 'a0',
            'amplitude_1': 'a1',
            'amplitude_2': 'a2',
        }
    ).rename_axis(['mip', 'dataset']).sort_index(key=lambda x: x.str.lower())


In [5]:
# CMIP6 models used in Tsutsui (2020)
sources_cmip6_grl_paper = [
    'BCC-CSM2-MR', 'BCC-ESM1', 'CanESM5', 'CESM2', 'CESM2-WACCM',
    'CNRM-CM6-1', 'CNRM-ESM2-1', 'GFDL-CM4', 'GISS-E2-1-G', 'GISS-E2-1-H',
    'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MIROC-ES2L', 'MRI-ESM2-0',
    'SAM0-UNICON', 'UKESM1-0-LL', 'EC-Earth3-Veg', 'CAMS-CSM1-0',
    'E3SM-1-0', 'NESM3', 'NorESM2-LM',
]

In [6]:
# Plot class customized for A4 landscape in PDF
class PlotPdf(PlotBase):
    def init_general(self, **kw):
        kw = kw.copy()

        paper_width = 297.
        paper_height = 210.
        mmpi = 25.4 # mm/inch

        kw['left'] = 1.5
        kw['top'] = 1.5
        width = kw['height'] * kw['aspect']
        kw['right'] = paper_width/mmpi - (
            2*width + kw['wspace'] + kw['left']
        )
        kw['bottom'] = paper_height/mmpi - (
            2*kw['height'] + kw['hspace'] + kw['top']
        )
        super().init_general(**kw)
        self.plot_space.mmpi = mmpi
        self.plot_space.paper_width = paper_width
        self.plot_space.paper_height = paper_height

    def get_xy_figure_title(self):
        """Return (x, y) in figure units for a figure title
        """
        plot_space = self.plot_space
        space = plot_space.kw_space
        paper_width_in  = plot_space.paper_width / plot_space.mmpi
        paper_height_in = plot_space.paper_height / plot_space.mmpi

        width = space['height'] * space['aspect']
        x = (
            space['left'] + width + 0.5*space['wspace']
        ) / paper_width_in
        y = (
            space['bottom'] + 2*space['height'] + space['hspace'] + 0.3
        ) / paper_height_in

        return x, y

    def savefig(self, path, **kw):
        """Wrapper for figure.savefig()
        """
        if isinstance(path, PdfPages):
            print('page [{}]'.format(path.get_pagecount() + 1))
            path.savefig(self.figure)
        else:
            super().savefig(path, **kw)

In [7]:
# Read CMIP data and determine common data ranges
min_n = []
max_n = []
min_t = []
max_t = []
gcm_data = {}

for (mip, dataset), parms in df_parms.iterrows():
    df = df_norm[mip].loc[dataset]
    gcm = {
        '4x_n': df.loc[('abrupt-4xCO2', 'rtnt')].dropna().values,
        '4x_t': df.loc[('abrupt-4xCO2', 'tas')].dropna().values,
        '1p_n': df.loc[('1pctCO2', 'rtnt')].dropna().values,
        '1p_t': df.loc[('1pctCO2', 'tas')].dropna().values,
    }
    gcm_data[dataset] = gcm

    min_n.append(min(gcm['4x_n'].min(), gcm['1p_n'].min()))
    max_n.append(max(gcm['4x_n'].max(), gcm['1p_n'].max()))
    min_t.append(min(gcm['4x_t'].min(), gcm['1p_t'].min()))
    max_t.append(max(gcm['4x_t'].max(), gcm['1p_t'].max()))

xlim = [min(min_t)-0.2, max(max_t)]
ylim = [min(min_n)-0.2, max(max_n)]

fmax = (df_parms['co2_alpha'] * df_parms['co2_beta'] * np.log(4.)).max()
fmax_reg = (df_parms['lambda_reg'] * df_parms['ecs_reg'] * 2).max()
tmax = (df_parms['co2_alpha'] * df_parms['co2_beta'] * np.log(4.) \
        / df_parms['lambda']).max()
tmax_reg = df_parms['ecs_reg'].max() * 2

xlim[1] = max(xlim[1], tmax, tmax_reg)
ylim[1] = max(ylim[1], fmax, fmax_reg)

In [8]:
forcing = RfCO2()
climate = IrmBase()
myplt = PlotPdf()

In [9]:
with PdfPages('image/fitting_results.pdf') as outpath:
    n = 0

    for (mip, dataset), parms in df_parms.iterrows():
        parms = df_parms.loc[(mip, dataset)]

        forcing.parms.update(
            alpha=parms['co2_alpha'],
            beta=parms['co2_beta'],
        )
        climate.parms.update(
            asj=parms[['a0', 'a1', 'a2']].values,
            tauj=parms[['tau0', 'tau1', 'tau2']].values,
            lamb=parms['lambda'],
        )

        names = {'var_n': 'rtnt', 'var_t': 'tas', 'dataset': dataset}
        plot_fitting(
            myplt, gcm_data[dataset], forcing, climate, names, parms.to_dict(),
        )
        ax = myplt(2)
        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)

        xpos, ypos = myplt.get_xy_figure_title()
        myplt.figure.text(
            xpos, ypos, f'{mip}, {dataset}',
            ha='center', va='bottom', fontsize='x-large',
        )

        if mip == 'CMIP5' or dataset in sources_cmip6_grl_paper:
            n += 1
            print('Figure S{}'.format(2+n), mip, dataset, end=' ')
        else:
            print('Figure added', mip, dataset, end=' ')

        myplt.savefig(outpath)
        myplt.close()

Figure S3 CMIP5 ACCESS1.0 page [1]
Figure S4 CMIP5 ACCESS1.3 page [2]
Figure S5 CMIP5 BCC-CSM1.1 page [3]
Figure S6 CMIP5 BNU-ESM page [4]
Figure S7 CMIP5 CanESM2 page [5]
Figure S8 CMIP5 CCSM4 page [6]
Figure S9 CMIP5 CNRM-CM5 page [7]
Figure S10 CMIP5 CSIRO-Mk3.6.0 page [8]
Figure S11 CMIP5 FGOALS-s2 page [9]
Figure S12 CMIP5 GFDL-CM3 page [10]
Figure S13 CMIP5 GFDL-ESM2G page [11]
Figure S14 CMIP5 GFDL-ESM2M page [12]
Figure S15 CMIP5 GISS-E2-H page [13]
Figure S16 CMIP5 GISS-E2-R page [14]
Figure S17 CMIP5 HadGEM2-ES page [15]
Figure S18 CMIP5 INM-CM4 page [16]
Figure S19 CMIP5 IPSL-CM5A-LR page [17]
Figure S20 CMIP5 IPSL-CM5B-LR page [18]
Figure S21 CMIP5 MIROC-ESM page [19]
Figure S22 CMIP5 MIROC5 page [20]
Figure S23 CMIP5 MPI-ESM-LR page [21]
Figure S24 CMIP5 MPI-ESM-MR page [22]
Figure S25 CMIP5 MPI-ESM-P page [23]
Figure S26 CMIP5 MRI-CGCM3 page [24]
Figure S27 CMIP5 NorESM1-M page [25]
Figure added CMIP6 ACCESS-CM2 page [26]
Figure added CMIP6 ACCESS-ESM1-5 page [27]
Figure 